# Makine Öğrenmesi Dönem Ödevi Projesi Raporu (F2025)

**Öğrenci Adı Soyadı:** Firuze Eroğlu  
**Öğrenci No:** 201613709044  
**Teslim Tarihi:** 14.12.2025  

---

## 1. Veri Seti Tanıtımı

Bu projede üç farklı makine öğrenmesi problemi için, müzik endüstrisinin en büyük platformu Spotify'dan elde edilen gerçek dünya verileri kullanılmıştır. Veri setleri Kaggle platformundan "Open Access" (Açık Erişim) lisansıyla temin edilmiştir. Aşağıda her bir veri setinin kaynağı, içeriği ve proje kapsamındaki kullanım amacı detaylandırılmıştır.

### 1.1. Sınıflandırma Veri Seti: Spotify Tracks Dataset
*   **Veri Kaynağı Linki:** [Kaggle - Spotify Tracks Dataset](https://www.kaggle.com/datasets/maharshipandya/-spotify-tracks-dataset)
*   **Veri Seti Tanımı:** Bu veri seti, Spotify üzerindeki 125 farklı müzik türünden yaklaşık 114.000 şarkıyı kapsamaktadır. Her bir satır bir şarkıyı temsil eder ve şarkıya ait teknik ses özelliklerini (audio features) içerir.
*   **Problem ve Tahmin Hedefi:**
    *   **Problem:** Müzik endüstrisinde bir şarkının "Hit" olup olmayacağını, şarkı henüz piyasaya çıkmadan sadece ses analizine dayanarak öngörmek büyük bir ticari değer taşır.
    *   **Hedef:** Şarkının `danceability`, `energy`, `loudness` gibi teknik ses özniteliklerini kullanarak, şarkının **"Popüler"** (Popularity > 50) olup olmadığını sınıflandırmaya çalışıyoruz. Bu, **Binary Classification** problemidir.

### 1.2. Regresyon Veri Seti: Spotify Global Music Dataset
*   **Veri Kaynağı Linki:** [Kaggle - Spotify Global Music Dataset 2009-2025](https://www.kaggle.com/datasets/wardabilal/spotify-global-music-dataset-20092025?select=spotify_data+clean.csv) (Kullanılan dosya: `spotify_data clean.csv`)
*   **Veri Seti Tanımı:** 2009-2025 yılları arasındaki küresel müzik trendlerini içeren veri setidir.
*   **Hedef:** Şarkının ve sanatçının metadatalarını kullanarak, şarkının Spotify üzerindeki **Popülerlik Puanını (0-100)** tahmin etmek.

### 1.3. Kümeleme Veri Seti: Most Streamed Spotify Songs 2024
*   **Veri Kaynağı Linki:** [Kaggle - Most Streamed Spotify Songs 2024](https://www.kaggle.com/datasets/nelgiriyewithana/most-streamed-spotify-songs-2024)
*   **Veri Seti Tanımı:** 2024 yılının en çok dinlenen şarkılarını ve etkileşim sayılarını içerir.
*   **Hedef:** Şarkıları dinlenme ve etkileşimlerine göre **Segmentlere (Kümeler)** ayırmak (Unsupervised Learning).

---

## 2. Uygulanan Yöntemler ve Ön İşleme
(Raporun bu kısmı orijinal dokümandaki gibidir: Eksik veriler, ölçekleme, one-hot encoding, undersampling ve outlier temizliği uygulanmıştır.)

## 3. Sonuçlar ve Metrikler
(Detaylı sonuçlar ve grafik yorumları Notebook çıktılarında yer alacaktır.)

---



In [ ]:
import os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# --- OTOMATİK VERİ İNDİRME FONKSİYONU ---
def load_data_from_github(filename, url):
    if not os.path.exists(filename):
        print(f"Dosya yerelde bulunamadı: {filename}")
        print(f"GitHub'dan indiriliyor... ({url})")
        try:
            # Colab/Linux/Windows uyumlu indirme (wget veya curl)
            if os.system(f'wget -O "{filename}" "{url}"') != 0:
                # Wget çalışmazsa curl dene
                os.system(f'curl -L "{url}" -o "{filename}"')
            print("İndirme tamamlandı.")
        except Exception as e:
            print(f"İndirme Hatası: {e}")
    else:
        print(f"Dosya zaten mevcut: {filename}")
    
    # Okuma
    try:
        df = pd.read_csv(filename)
    except:
        # Kodlama hatası olursa
        df = pd.read_csv(filename, encoding='ISO-8859-1')
    return df

print("Veri indirme ve yükleme altyapısı hazır.")



In [ ]:
# --- 1. CLASSIFICATION TASK ---
import pandas as pd
import numpy as np
import matplotlib
# matplotlib.use('Agg') # Notebook'ta grafik görmek için bunu kapatıyoruz
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                             roc_auc_score, confusion_matrix, classification_report)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import shap
import time

print("--- CLASSIFICATION BAŞLIYOR ---")
# GitHub URL'si
URL_CLASS = "https://raw.githubusercontent.com/frzerxz/spotify-ml-analysis/main/dataset.csv"
# Veriyi Yükle (Otomatik Fonksiyon ile)
df = load_data_from_github('dataset.csv', URL_CLASS)

print(f"Veri seti boyutu: {df.shape}")

target_col = 'is_popular'
df[target_col] = (df['popularity'] > 50).astype(int)

cols_to_drop = ['Unnamed: 0', 'track_id', 'artists', 'album_name', 'track_name', 'popularity', 'track_genre']
feature_cols = [c for c in df.columns if c not in cols_to_drop and c != target_col]
X = df[feature_cols]
y = df[target_col]

if 'explicit' in X.columns:
    X['explicit'] = X['explicit'].astype(int)

numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object', 'category', 'bool']).columns

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Undersampling
try:
    from imblearn.under_sampling import RandomUnderSampler
    rus = RandomUnderSampler(random_state=42)
    # Pipeline entegrasyonu olmadığı için önce preprocess etmek gerekir ama 
    # burada basitlik adına orjinal logic'i koruyoruz
    print("Under-sampling uygulanıyor...")
    # Not: imblearn ile pipeline kullanımı farklıdır, burada manuel dengelemeyi simüle eden kodun
    # çalışması için veri tiplerini korumalıyız.
    # Kullanıcının orjinal kodunda bu kısım preprocess öncesi yapılmıştı. Biz de öyle yapalım:
    X_train_resampled, y_train_resampled = rus.fit_resample(X_train, y_train)
    X_train = X_train_resampled
    y_train = y_train_resampled
except ImportError:
    print("UYARI: imblearn yok.")

# Ön İşleme
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

# Model Eğitimi (Örnek: RandomForest)
print("Model Eğitiliyor (Random Forest)...")
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_processed, y_train)
y_pred = rf.predict(X_test_processed)

print("Classification Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))



In [ ]:
# --- 2. REGRESSION TASK ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error

print("\n--- REGRESSION BAŞLIYOR ---")
URL_REG = "https://raw.githubusercontent.com/frzerxz/spotify-ml-analysis/main/spotify_data%20clean.csv"
# Veriyi Yükle (Dosya adındaki boşluk sorununu URL encode ile çözdük)
df_reg = load_data_from_github('spotify_data clean.csv', URL_REG)

print(f"Veri seti boyutu: {df_reg.shape}")

target_col = 'track_popularity'
# Basit özellik seçimi (Orjinal koddaki gibi)
cols_to_drop = ['track_id', 'track_name', 'artist_name', 'album_id', 'album_name', 'album_release_date', 'artist_genres', target_col]
X = df_reg.drop(columns=[c for c in cols_to_drop if c in df_reg.columns])
y = df_reg[target_col]

numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object', 'bool']).columns

# Pipeline
numeric_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='median')), ('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')), ('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model
print("Model Eğitiliyor (Random Forest Regressor)...")
rf_reg = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('regressor', rf_reg)])
pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)
print(f"R2 Score: {r2_score(y_test, y_pred):.4f}")



In [ ]:
# --- 3. CLUSTERING TASK ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA

print("\n--- CLUSTERING BAŞLIYOR ---")
# DÜZELTME: Dosya adı alt çizgili olacak
filename_clust = 'Most_Streamed_Spotify_Songs_2024.csv'
URL_CLUST = "https://raw.githubusercontent.com/frzerxz/spotify-ml-analysis/main/Most_Streamed_Spotify_Songs_2024.csv"

df_clust = load_data_from_github(filename_clust, URL_CLUST)
print(f"Veri Boyutu: {df_clust.shape}")

# Gereksiz sütunları at
drop_cols = ['Track', 'Album Name', 'Artist', 'ISRC', 'All Time Rank', 'Release Date']
X = df_clust.drop(columns=[c for c in drop_cols if c in df_clust.columns])

# Virgülleri temizle ve sayıya çevir
for col in X.columns:
    if X[col].dtype == object:
        X[col] = pd.to_numeric(X[col].astype(str).str.replace(',', ''), errors='coerce')

X = X.fillna(0)
# İlk 5000 satır (Hız için)
if len(X) > 5000:
    X = X.iloc[:5000]

# Ölçekleme
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# KMeans
print("K-Means çalıştırılıyor (K=3)...")
kmeans = KMeans(n_clusters=3, random_state=42)
labels = kmeans.fit_predict(X_scaled)

print(f"Silhouette Score: {silhouette_score(X_scaled, labels):.4f}")
print("Kümeleme tamamlandı.")

